# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome()

# Passo 1: Obter a cotação do Dólar;
browser.get('https://www.google.com')

browser.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys('Cotação Dólar')
browser.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys(Keys.ENTER)
cot_dol = browser.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cot_dol)

# Passo 2: Obter a cotação do Euro;
browser.get('https://www.google.com')

browser.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys('Cotação Euro')
browser.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys(Keys.ENTER)
cot_euro = browser.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cot_euro)

# Passo 3: Obter a cotação do Ouro;
browser.get('https://www.melhorcambio.com/ouro-hoje')
cot_ouro = browser.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cot_ouro = cot_ouro.replace(',','.')

print(cot_ouro)

browser.quit()

5.5331
6.331799999999999
324.80


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [23]:
# Passo 4: Importar a base e atualizar as cotações;
import pandas as pd

db = pd.read_excel('Produtos.xlsx')
display(db)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [26]:
# Passo 5: Calcular os novos preços e salvar/exportar a base de dados.

db.loc[db['Moeda'] == 'Dólar', 'Cotação'] = float(cot_dol)
db.loc[db['Moeda'] == 'Euro', 'Cotação'] = float(cot_euro)
db.loc[db['Moeda'] == 'Ouro', 'Cotação'] = float(cot_ouro)

# Atualizar as colunas de preço e venda:
db['Preço de Compra'] = db['Preço Original']*db['Cotação']
db['Preço de Venda'] = db['Preço de Compra']*db['Margem']

display(db)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.5331,5533.044669,1.40,7746.262537
1,Carro Renault,4500.00,Euro,6.3318,28493.100000,2.00,56986.200000
2,Notebook Dell,899.99,Dólar,5.5331,4979.734669,1.70,8465.548937
3,IPhone,799.00,Dólar,5.5331,4420.946900,1.70,7515.609730
4,Carro Fiat,3000.00,Euro,6.3318,18995.400000,1.90,36091.260000
5,Celular Xiaomi,480.48,Dólar,5.5331,2658.543888,2.00,5317.087776
6,Joia 20g,20.00,Ouro,324.8000,6496.000000,1.15,7470.400000


### Agora vamos exportar a nova base de preços atualizada

In [28]:
db.to_excel('Produtos atualizados.xlsx', index = False)